# 0710012 何權祐 HW3

In [309]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib3
import re
from io import StringIO

%matplotlib inline

# Data Input

In [310]:
data = pd.read_csv(r'Ionosphere/ionosphere.data',sep=',',header=None)
data

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,g
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,b
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,g
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,b
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,1,0,0.83508,0.08298,0.73739,-0.14706,0.84349,-0.05567,0.90441,-0.04622,...,-0.04202,0.83479,0.00123,1.00000,0.12815,0.86660,-0.10714,0.90546,-0.04307,g
347,1,0,0.95113,0.00419,0.95183,-0.02723,0.93438,-0.01920,0.94590,0.01606,...,0.01361,0.93522,0.04925,0.93159,0.08168,0.94066,-0.00035,0.91483,0.04712,g
348,1,0,0.94701,-0.00034,0.93207,-0.03227,0.95177,-0.03431,0.95584,0.02446,...,0.03193,0.92489,0.02542,0.92120,0.02242,0.92459,0.00442,0.92697,-0.00577,g
349,1,0,0.90608,-0.01657,0.98122,-0.01989,0.95691,-0.03646,0.85746,0.00110,...,-0.02099,0.89147,-0.07760,0.82983,-0.17238,0.96022,-0.03757,0.87403,-0.16243,g


# Data Processing

In [311]:
from sklearn.utils import shuffle
data = shuffle(data)
feature = data.drop(columns=34).reset_index(drop=True)
Label = pd.DataFrame(data[34]).reset_index(drop=True)
Label

,34
0,b
1,b
2,g
3,b
4,g
...,...
346,g
347,g
348,g
349,b


# SVM

In [312]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    feature.values,Label.values.ravel(),train_size=0.7)

poly_grid = {'degree': [3,4,5], 
                'coef0':[1,10,100],
                'gamma': [1,0.1,0.001],
                'kernel': ['poly']},
rbf_grid = {'gamma': [1,0.1,0.001],
                'kernel': ['rbf']}
poly_ker = GridSearchCV(SVC(),poly_grid,cv=5)
rbf_ker = GridSearchCV(SVC(),rbf_grid,cv=5)

poly_ker.fit(X_train,y_train)
rbf_ker.fit(X_train,y_train)
print("Polynomial best hyperparameter pair:")
print(poly_ker.best_params_)
print("\nRBF best hyperparameter pair:")
print(rbf_ker.best_params_)

# poly_m = SVC()
# poly_m.set_params(**poly_ker.best_estimator_)
# rbf_m = SVC()
# rbf_m.set_params(**rbf_ker.best_estimator_)

clf = [make_pipeline(StandardScaler(), SVC(kernel='linear')),
        make_pipeline(StandardScaler(), SVC(**poly_ker.best_params_)),
        make_pipeline(StandardScaler(), SVC(**rbf_ker.best_params_))]

clf[0].fit(X_train,y_train)
clf[1].fit(X_train,y_train)
clf[2].fit(X_train,y_train)
pass

Polynomial best hyperparameter pair:
{'coef0': 1, 'degree': 3, 'gamma': 0.1, 'kernel': 'poly'}

RBF best hyperparameter pair:
{'gamma': 0.1, 'kernel': 'rbf'}


# Result

In [313]:
predict=[]
accuracy=[]
precision=[]
recall=[]
Confusion_mat=[]

for i in range(3):
    predict.append(clf[i].predict(X_test))
    accuracy.append(clf[i].score(X_test, y_test))
    precision.append(precision_score(y_test.flatten(),predict[i],average=None,labels=['b', 'g']))
    recall.append(recall_score(y_test.flatten(),predict[i],average=None,labels=['b', 'g']))
    Confusion_mat.append(confusion_matrix(y_test.flatten(),predict[i],labels=['b', 'g']))

print(accuracy)

[0.8490566037735849, 0.8584905660377359, 0.9245283018867925]


### Accuraay & Precision & Recall

In [314]:
from tabulate import tabulate
print("Accuracy:")
print(tabulate([[accuracy[0],accuracy[1],accuracy[2]]],headers=['Linear','Poly','RBF'],tablefmt='grid'))

print('\nPrecision:')
p=[]
precision = np.array(precision).transpose()
for n in precision:
    p.append([m for m in n])

tabulate(p,headers=['Label','Linear','Poly','RBF'],tablefmt='grid')
p[0].insert(0,'b')
p[1].insert(0,'g')
print(tabulate(p,headers=['Label','Linear','Poly','RBF'],tablefmt='grid'))

print('\nRecall(Sensitivity):')
r=[]
recall = np.array(recall).transpose()
for n in recall:
    r.append([m for m in n])

tabulate(r,headers=['Label','Linear','Polynomial','RBF'],tablefmt='grid')
r[0].insert(0,'b')
r[1].insert(0,'g')
print(tabulate(r,headers=['Label','Linear','Polynomial','RBF'],tablefmt='grid'))

# print(Confusion_mat)

Accuracy:
+----------+----------+----------+
|   Linear |     Poly |      RBF |
+==========+==========+==========+
| 0.849057 | 0.858491 | 0.924528 |
+----------+----------+----------+

Precision:
+---------+----------+----------+----------+
| Label   |   Linear |     Poly |      RBF |
+=========+==========+==========+==========+
| b       |  0.92    | 0.923077 | 0.891892 |
+---------+----------+----------+----------+
| g       |  0.82716 | 0.8375   | 0.942029 |
+---------+----------+----------+----------+

Recall(Sensitivity):
+---------+----------+--------------+----------+
| Label   |   Linear |   Polynomial |      RBF |
+=========+==========+==============+==========+
| b       | 0.621622 |     0.648649 | 0.891892 |
+---------+----------+--------------+----------+
| g       | 0.971014 |     0.971014 | 0.942029 |
+---------+----------+--------------+----------+


### Confusion Matrix

In [315]:
tp = ['Linear','Poly','RBF']
for i  in range(3):
    print("%s confusion_matrix:"%tp[i])
    print(tabulate(confusion_matrix(y_test,clf[i].predict(X_test),labels=['b','g']),tablefmt='grid'),end='\n\n')

Linear confusion_matrix:
+----+----+
| 23 | 14 |
+----+----+
|  2 | 67 |
+----+----+

Poly confusion_matrix:
+----+----+
| 24 | 13 |
+----+----+
|  2 | 67 |
+----+----+

RBF confusion_matrix:
+----+----+
| 33 |  4 |
+----+----+
|  4 | 65 |
+----+----+



# Comparion and Conclusion
After running the three different models several times, I observe that the performance is often Linear < Poly < RBF kernel. However, I think it's a bit unfair to compare the Linear kernel to the other two kernel model, as Poly and RBF can try several differnt hyperparameters and choose the one with the best performance.

We can see from the confusion matrix that the determined difference of the three kernel is the true positive part, as the three kernel have nearly some number of cases in their true ngative part.

Also, we can conclude from that the result that RBF is the best kernel that fits this data set.

# Question
>Show the average performance of K-fold cross-validation of parameter search in tables for each kernel.

In [316]:
means_r = rbf_ker.cv_results_["mean_test_score"]
print('RBF kernel performance:')
print(tabulate([['Result',means_r[0],means_r[1],means_r[2]]],headers=['gamma','1','0.1','0.001'],tablefmt='grid'))


print('\nPoly kernel performance:')
means_p = poly_ker.cv_results_["mean_test_score"]
ranks_p = poly_ker.cv_results_["rank_test_score"]
table=[]
i=0
for params in poly_ker.cv_results_["params"]:
    table.append([params['coef0'],params['degree'],params['gamma'],means_p[i],ranks_p[i]])
    i+=1
# print(i)
rbf = pd.DataFrame(table)
rbf.columns=['coef0','degree','gamma','mean_test_score','rank_test_score']
rbf=rbf.sort_values(by=['rank_test_score']).reset_index(drop=True)

rbf

RBF kernel performance:
+---------+----------+----------+----------+
| gamma   |        1 |      0.1 |    0.001 |
+=========+==========+==========+==========+
| Result  | 0.910204 | 0.938776 | 0.636735 |
+---------+----------+----------+----------+

Poly kernel performance:


,coef0,degree,gamma,mean_test_score,rank_test_score
0,1,3,0.100,0.918367,1
1,1,4,0.100,0.906122,2
2,1,4,1.000,0.889796,3
3,1,5,0.100,0.885714,4
4,10,5,0.001,0.885714,4
5,100,3,0.001,0.881633,6
6,1,5,1.000,0.881633,7
7,1,3,1.000,0.877551,8
8,10,4,0.001,0.873469,9
9,10,5,1.000,0.873469,10
